1. Entendimento do problema
2. Importações e configurações do ambiente
3. Estatísticas descritivas
4. Análise, tratamento e transformação de variáveis
5. Modelo linear (baseline) e suas premissas
6. Estimando os coeficientes e realizando testes de hipótese
7. Acurácia do modelo
8. Decidindo variáveis importantes (Variable Selection)
9. Intervalos de confiança e de predição
10. Revisitando as premissas do modelo
11. Problemas potenciais

## 1. Entendimento do problema

O primeiro passo consiste em entender/formatar o problema de modo que modelá-lo através de um modelo linear faça sentido para responder as perguntas. Embora simples, os modelos lineares são muito eficientes e altamente interpretáveis em diversas situações reais. As principais perguntas que podem ser respondidas em problemas como esse são:

[formular depois]
* Há alguma relação entre as variáveis preditoras $X_1,X_2 \cdots X_p$ e a resposta $y$?
* Quão forte é a relação entre as variáveis preditoras e a resposta?
* Qual das variáveis preditoras está relacionada com a resposta?

## 2. Importações e configurações do ambiente

Nessa seção, vamos importar pacotes, classes e até mesmo bibliotecas inteiras que nos auxiliem na construção do programa

In [ ]:
# Estruturação e manipulação dos dados
import numpy  as np
import pandas as pd

# Visualizações
import matplotlib.pyplot as plt

# Estatística
import statsmodels.api as sm

# Modelos
from sklearn.model_selection import train_test_split

## 3. Leitura dos dados
Carregamos o arquivo com os dados para uma estrutura compatível (*e.g.*, `pd.DataFrame`) onde serão analisados e processados. Também serão exibidas as primeiras observações (linhas) da tabela usando o método `.head()`

In [ ]:
path_arquivo = r'.\datasets\Advertising.csv'
dados = pd.read_csv(path_arquivo)

In [ ]:
# Primeiras observações
dados.head()

In [ ]:
# Removendo a primeira coluna com os índices desnecessários
dados.drop('Unnamed: 0', axis=1, inplace=True)

### Separação em conjunto de treino e teste

In [ ]:
X = dados.iloc[:,:-1]
y = dados.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. Distribuição das variáveis quantitativas
Serão exploradas algumas estatísticas descritivas das variáveis numéricas; essa etapa nos auxilia a ter uma visão geral da **distribuição** dessas variáveis. Também será analisada a estrutrura geral do conjunto de dados, como quantidade de linhas e colunas e ocorrência de valores duplicados, nulos ou ausentes.

In [ ]:
var_num = dados.select_dtypes(exclude='object').columns

In [ ]:
# Estatísticas das variáveis quantitativas
X_train.describe().T

In [ ]:
# Distribuições das variáveis numéricas
p_num = len(var_num)
fig, axes = plt.subplots(2, p_num, figsize=(p_num*5,5), gridspec_kw={'height_ratios':[2,1],'hspace':0.01})
for i,var in enumerate(var_num):
    axes[0][i].hist(dados.loc[X_train.index][var],histtype='step',density=True)
    axes[0][i].set_title(var)
    axes[1][i].boxplot(dados.loc[X_train.index][var],widths=[.5],labels=[var],positions=[i],showfliers=False,vert=False)
    axes[1][i].scatter(x=dados.loc[X_train.index][var],y=np.zeros_like(dados.loc[X_train.index][var])+np.random.normal(i,0.05,len(dados.loc[X_train.index][var])),alpha=0.3,s=5)
    axes[1][i].axis('off')
plt.show()

In [ ]:
# Estatísticas gerais e estrutura dos dados
dados.loc[X_train.index].info()

In [ ]:
# Valores duplicados
print(f'Observações duplicadas: {dados.loc[X_train.index].duplicated().sum()}')

In [ ]:
print(f'Observações ausentes: \n{dados.loc[X_train.index].isnull().sum()}')

## 5. Análise, tratamento e transformação de variáveis
Esta seção é dedicada ao tratamento e transformação dos dados, buscando trazer integridade e qualidade para as análises posteriores. Destaco as mais comuns como sendo:

* Preenchimento de valores ausentes;
* Remoção de valores duplicados;
* Exclusão de variáveis com poucas observações;
* Ajuste de escalas;
* Criação de variáveis auxiliares;
* Correção de erros de preenchimento em geral;

Começaremos analisando a correlação entre as variáveis

In [ ]:
dados.loc[X_train.index].corr()

Em seguida, vamos visualizar um gráfico de dispersão entre elas

In [ ]:
# Dispersão contra variável alvo
p_num = len(var_num)
fig, axes = plt.subplots(1, p_num-1, figsize=(p_num*5,5),sharey=True)
for i,var in enumerate(var_num):
    if var != 'sales':
        axes[i].scatter(dados.loc[X_train.index][var],dados.loc[X_train.index]['sales'],alpha=0.5)
        axes[i].set_xlabel(var,fontsize=14,loc='right')
        axes[i].set_title('Sales',loc='left',fontsize=14)
        axes[i].grid(alpha=0.5)
plt.show()

> Dos gráficos acima, de fato verificamos que é possível aproximar a relação entre as variáveis **TV** e **radio** com **sales** por meio de uma função linear. O mesmo já não pode ser dito quanto a variável **newspaper**

## 6. Modelo linear (*baseline*) e suas premissas

O modelo linear multivariado consiste em ajustar o conjunto de dados de treino a uma função que toma a seguinte forma:

$$
y = \hat\beta_0 + \hat\beta_1 X_1 + \cdots + \hat\beta_p X_p \qquad \therefore \qquad \mathbf{\hat y} = \mathbf{X \hat\beta}
$$

Onde:
* $\hat\beta_j$ é o valor estimado para o coeficiente $\beta_j$, obtido usando método dos mínimos quadrados

Matricialmente, a solução de mínimos quadrados acima consiste em:

$$
\mathbf{\hat\beta} = \mathbf{(XX^{\top})^{-1}X^{\top}y}
$$

### Treinando o modelo

Vamos definir a matriz $\mathbf{X}$ como sendo o subconjunto dos dados contendo somente as variáveis numéricas e $y$ como sendo o vetor coluna com a resposta

In [ ]:
# Ajustando os dados de treino ao modelo linear
X_train = sm.add_constant(X_train)    # Adicionamos o intercepto
modelo = sm.OLS(y_train, X_train).fit()

## 7. Estimando os coeficientes e realizando testes de hipótese

In [ ]:
round(estatisticas_parametros(modelo),4)

Os valores estimados para os coeficientes se encontram na tabela acima. Junto a eles, também temos acesso ao desvio padrão desses estimadores e seus correspondentes *p-values*. De posse desses resultados, podemos concluir que:

* **Intercepto**<br>
Na ausência de qualquer investimento em mídias na TV, Rádio ou Jornal, espera-se que a as vendas estejam no intervalo de confiança abaixo

    $\hat\beta_0 = 2.94 \pm 0.31$

* **TV**<br>
Fixado os investimentos em Rádio e Jornal, espera-se que um incremento de 1,000 dolares em propaganda na TV esteja associado a um aumento nas vendas de

    $\hat\beta_1 = 45.8 \pm 1.4$

* **Rádio**<br>
Fixado os investimentos em TV e Jornal, espera-se que um incremento de 1,000 dolares em propaganda na Rádio esteja associado a um aumento nas vendas de

    $\hat\beta_2 = 188.5 \pm 8.6$

* **Jornal**<br>
Fixado os investimentos em TV e Rádio, espera-se que um incremento de 1,000 dolares em propaganda em Jornal esteja associado a um aumento nas vendas de

    $\hat\beta_3 = -1 \pm 5.9$

> Esse intervalo seguramente cobre o zero em 95% das vezes. Sendo assim, estamos decididos que não há significância estatística

Das conclusões acima, irei responder algumas perguntas:

### Há alguma relação entre a variável alvo $y$ e os preditores $X$?

Para averiguar se os coeficientes estimados são estatisticamente significantes (i.e., existe relação entre $X$ e $y$), precisamos calcular a *F-Statistic* do modelo

$$
\text{F} = \frac{(\text{TSS-RSS})/p}{\text{RSS}/(n-p-1)}
$$

Caso as premissas do modelo linear estiverem corretas, esperamos que:
* $\mathbb{E}[\text{RSS}/(n-p-1)] = \sigma^2$
Dai, assumindo que a hipótese nule $H_0$ esteja correta:
* $\mathbb{E}[(\text{TSS-RSS})/p] = \sigma^2$

Portanto, se **não houver** nenhuma relação entre $X$ e $y$, esperamos que $\text{F} \approx 1$. Se houver alguma relação, esperamos que $\text{F} \gg 1$

In [ ]:
print(f'F-statistic: {round(modelo.fvalue,2)}')
print(f'F p-value  : {modelo.f_pvalue:.3f}')

No modelo considerado, obtivemos $\text{F} = 422.2 \gg 1$, e sua probabilidade associada muito menor que 5%, indicando forte relação entre a resposta e alguma variável preditora.

## 8. Desempenho do modelo

O modelo fora treinado com um conjunto de dados. Sua acurácia será determinada quando, a partir do modelo ajustado, compararmos as respostas preditas com um novo conjunto de dados (teste) ainda desconhecido pelo modelo. As principais métricas de avaliação de modelo são o RSE e R<sup>2</sup>

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Adicionamos a coluna com 1's com o intercepto
X_test = sm.add_constant(X_test)
y_pred = modelo.predict(X_test)

In [ ]:
print(f'R2: {round(r2_score(y_test,y_pred),3)}')
print(f'RSE: {round(mean_squared_error(y_test,y_pred),3)}')

Dos resultados acima, concluimos que:

* ~90% da variabilidade da resposta $y$ é explicada pelo modelo linear (*i.e*, pelas variáveis preditoras)
* Na média, as predições variam cerca de 2.880 unidades em relação ao valor real

### Visualizando as predições

In [ ]:
visualizar_predicoes(X_test, y_test, modelo)

### Funções auxiliares

In [ ]:
def estatisticas_parametros(model):
    summary = {
        'Coefficient': model.params,
        'Std. error': model.bse,
        't-statistic': model.tvalues,
        'p-value': model.pvalues
    }
    
    # Criando a tabela em DataFrame
    results_table = pd.DataFrame(summary)
    results_table.index.name = 'Variable'
    
    # Exibindo a tabela
    return results_table

In [ ]:
def visualizar_predicoes(X_test, y_test, modelo):

    # Realiza as predições
    y_pred = modelo.predict(X_test)
    
    # Armazena os coeficientes/parametros do modelo
    beta_0 = modelo.params['const']
    betas  = modelo.params.iloc[1:].values

    # Criar gráficos de dispersão para cada preditor
    fig, axes = plt.subplots(1, X_test.shape[1]-1, figsize=(15, 5),sharey=True)
    
    for i, ax in enumerate(axes):
        
        # Pontos de teste e predições
        X = X_test.iloc[:,i+1]             # Desprezamos a coluna com intercepto
        ax.scatter(X, y_test, label="Teste", alpha=0.7, color='None', edgecolor='tab:green')
        ax.scatter(X, y_pred, label="Preditos", alpha=0.7,marker='x',color='tab:red')
        
        # Reta da regressão específica para o preditor
        x_range = np.linspace(X.min(), X.max(), 100)
        y_range = beta_0 + betas[i] * x_range
        ax.plot(x_range, y_range, color="k")
        ax.set_xlim(0,)
        ax.set_xlabel(X.name)
        ax.grid(alpha=0.5)
        
    plt.legend()
    plt.show()